In [ ]:
#@title Everything in Part 1 and Part 10 copied over

!pip install -U -q PyDrive
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from collections import defaultdict

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam, RMSprop
from keras.utils import plot_model

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

stockData = drive.CreateFile({'id':"1Gtf_20aW4H81JRwjvdKozlF49sXpZzID"}) 
stockData.GetContentFile('stock_data.csv')
cleaned_data = pd.read_csv('stock_data.csv')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In the previous part, we found out that the SVM performed better among all of the different models. We then combined the models to hopefully determine a better outcome, but it still performed worse than SVM. Hence, we decided to test it on other applications to answer 2 questions:

- Does SVM still perform better than combinations of models
- Is Sentiment Analysis a good indicator for change in stock price

In [ ]:
'''
We fit our models with our sentiment analysis dataset. We arbitrarily
chose a neural network as the combination since it performed well in
our analysis, and it best suits our purpose of using linear combinations.
'''
SVM_Model = SVC(C=1.25)
Combination_Model = MLPClassifier()

We then start accessing our new dataset:

In [ ]:
AppleNewsStock = drive.CreateFile({'id':"11AVKgIcGx3Q5_PAqpcfj-JxzkUL14AU_"}) 
AppleNewsStock.GetContentFile('AppleNewsStock.csv')  
AppleNews = pd.read_csv('AppleNewsStock.csv')
AppleNews = AppleNews.dropna()
AppleNews.head()

,Date,Open,High,Low,Close,Adj Close,Volume,News
0,2006-12-01,13.114285,13.190000,12.871428,91.320000,13.045714,198769900,WHAT'S ON TONIGHT : 8 P.M. (TLC) ASHLEY JUDD A...
1,2006-12-04,13.125714,13.150000,12.928572,91.120003,13.017143,177384200,More on Housing Prices : The broadest governme...
3,2006-12-06,12.948571,13.055715,12.810000,89.830002,12.832857,159546100,Honoring R.W. Apple in Words and Food : About ...
4,2006-12-07,12.861428,12.928572,12.414286,87.040001,12.434286,251206900,"Homebuilders, and Worries Over Jobs, Lead a De..."
5,2006-12-08,12.461429,12.770000,12.428572,88.259995,12.608571,196069300,"Homebuilders, and Worries Over Jobs, Lead a De..."


This has a lot of data that we don't need, which we will just remove.

In [ ]:
df = AppleNews.drop(['Date','Open','High','Low','Close','Volume'], axis=1)
df.head()

,Adj Close,News
0,13.045714,WHAT'S ON TONIGHT : 8 P.M. (TLC) ASHLEY JUDD A...
1,13.017143,More on Housing Prices : The broadest governme...
3,12.832857,Honoring R.W. Apple in Words and Food : About ...
4,12.434286,"Homebuilders, and Worries Over Jobs, Lead a De..."
5,12.608571,"Homebuilders, and Worries Over Jobs, Lead a De..."


Adj Close represents price of the Apple stock at the end of the day (adjusted so they are relative to each other). The News is the top headline of any news relating to Apple. We shall then polish the data as we did before:

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def remove_tags(string):
    result = re.sub('','',string)          #remove HTML tags
    result = re.sub('https://.*','',result)   #remove URLs
    result = re.sub(r'[^\w'+""+']', ' ',result) #remove non-alphanumeric characters
    result = re.sub(" \d+", " ", result) #remove numbers
    result = result.lower() #make all letters lowercase
    return result

def lemmatize_text(text): #Turns all text to basic form (like past test to present tense, removing the -ing in verbs)
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
df['News']= df['News'].apply(lambda cw : remove_tags(cw)) 
df['News'] = df['News'].apply(lemmatize_text)
cleaned_data2 = df
cleaned_data2.head()

,Adj Close,News
0,13.045714,what s on tonight p m tlc ashley judd and yout...
1,13.017143,more on housing price the broadest government ...
3,12.832857,honoring r w apple in word and food about of r...
4,12.434286,homebuilder and worry over job lead a decline ...
5,12.608571,homebuilder and worry over job lead a decline ...


We then want to determine whether the prices of the stock rise or fall:

In [ ]:
cleaned_data2['Rise'] = cleaned_data2['Adj Close'] - cleaned_data2['Adj Close'].shift(-1)
cleaned_data2['Rise'] = cleaned_data2['Rise'].apply(lambda x: 1 if x>=0 else -1)
cleaned_data2.head()

,Adj Close,News,Rise
0,13.045714,what s on tonight p m tlc ashley judd and yout...,1
1,13.017143,more on housing price the broadest government ...,1
3,12.832857,honoring r w apple in word and food about of r...,1
4,12.434286,homebuilder and worry over job lead a decline ...,-1
5,12.608571,homebuilder and worry over job lead a decline ...,-1


In [ ]:
cleaned_data2.rename(columns={'News': 'Text', 'Rise': 'Sentiment'}, inplace=True)
final_data = pd.concat((cleaned_data,cleaned_data2.iloc[:,1:]))
final_data.head()

,Text,Sentiment
0,kicker on my watchlist xide tit soq pnk cpw bp...,1
1,user aap movie return for the fea geed indicat...,1
2,user i d be afraid to short amzn they are look...,1
3,mnta over,1
4,oi over,1


We can then start analyzing:

In [ ]:
Xa = final_data['Text']
ya = final_data['Sentiment']
vec = CountVectorizer(max_features=4000)
Xa = vec.fit_transform(Xa).toarray()

pca = PCA(n_components=256)
Xa = pca.fit_transform(Xa)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(Xa, ya, test_size=0.2, random_state=0)

SVM_Model.fit(X_train,y_train)
y_pred = SVM_Model.predict(X_test)
print("Accuracy of SVM: {}".format(accuracy_score(y_test,y_pred)))

Accuracy of SVM: 0.6383240911891559


In [16]:
modelList = []
modelList.append(("KNN",KNeighborsClassifier(n_neighbors=30)))
modelList.append(("Linear Regression",RidgeClassifier(alpha=0)))
modelList.append(("Logistic Regression",LogisticRegression(C=0.45)))
modelList.append(("Neural Network", MLPClassifier(hidden_layer_sizes=(5, 5), activation='logistic', alpha=0.01, learning_rate_init=0.001)))
modelList.append(("Naive Bayes",BernoulliNB()))
modelList.append(("Gaussian Discriminant Analysis",GaussianNB()))
modelList.append(("SVM",SVC(C=1.25)))
modelList.append(("Decision Trees",DecisionTreeClassifier(criterion = "entropy", min_samples_split = 8)))

ls = []
for name, model in modelList:
    model.fit(Xa,ya)
    y_pred = model.predict(Xa)
    ls.append(y_pred)

X2 = pd.DataFrame(np.transpose(ls))
y2 = ya

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=40)

Combination_Model.fit(X2_train,y2_train)
y2_pred = Combination_Model.predict(X2_test)
print("Accuracy of Combined Model: {}".format(accuracy_score(y2_test,y2_pred)))

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy of Combined Model: 0.9864448552064079


Wow! This implies that a combined model has a much higher accuracy than strictly using SVM. This is most likely because the combined models accounts for other bias in the news that is not directly seen using SVM. This also implies that there is a high correlation between the sentiment of a news header as well as the change in stock price!